In [43]:
import pickle
import json

In [44]:
def load_collection_data_frames(filename='collection_data_frames.pkl'):
    with open(filename, 'rb') as file:
        data_frames = pickle.load(file)
    return data_frames

collection_data_frames = load_collection_data_frames()



In [45]:
collection_data_frames.keys()

dict_keys(['Calendar Items', 'Business Service Categories', 'Workspaces', 'Learning Pathway', 'Ticket Order', 'Business Types', 'Event Types', 'Ventures', 'Notes', 'PITCH Categories', 'Folder Types', 'Network Categories', 'Stages', 'Documents', 'Restaurants', 'Tasks', 'Accounts', 'Task Progress status', 'Shopping List Items', 'Contacts', 'Platforms', 'Chats', 'Portfolio Items', 'Tags', 'Enterprises', 'Vendor Types', 'Media', 'AI Chat Messages', 'Announcements', 'User Industry Points', 'Skills', 'Memory Media', 'Event NotesUpdates', 'Space Features', 'Communities', 'Conversations', 'Status Log', 'Chat History', 'Events', 'Venture Media', 'Statuses', 'Service Requests', 'Venture Team Members', 'Ticket Options', 'User Subcategory Points', 'Industry Subcategories', 'Discount Codes', 'Resources', 'Receipts', 'Discount Types', 'Contact Types', 'Business Services', 'Rental Rates', 'Message Attachment Types', 'Calendar Item Types', 'First Contact', 'App Photos', 'PITCH Subcategories', 'Objecti

In [46]:
# Function to check if a value is a list of numbers
def is_list_of_numbers(value):
    if isinstance(value, list) and all(isinstance(item, (int, float)) for item in value):
        return True
    return False

# Step 1: Create a new dictionary where values are lists of column names that have at least one row containing a list of numbers
filtered_collection_columns = {}

for collection_name, df in collection_data_frames.items():
    filtered_columns = []
    for column in df.columns:
        if df[column].apply(is_list_of_numbers).any():
            filtered_columns.append(column)
    filtered_collection_columns[collection_name] = filtered_columns

# Step 2: Find collection names that match column names
collection_names = set(collection_data_frames.keys())

# Step 3: Create a new dictionary where values are lists of columns without matching collection names
unique_columns_dict = {}

for collection_name, columns in filtered_collection_columns.items():
    unique_columns = [col for col in columns if col not in collection_names]
    unique_columns_dict[collection_name] = unique_columns

# Print the final dictionary for verification
print(unique_columns_dict)

{'Calendar Items': ['Event', 'Creating User', 'Users All', 'Calendar Item Type', 'Task'], 'Business Service Categories': [], 'Workspaces': [], 'Learning Pathway': ['User'], 'Ticket Order': ['Event', 'Ticket Option', 'Discount Code'], 'Business Types': [], 'Event Types': ['Mirror Tag'], 'Ventures': ['Users All', 'Business Type', 'Users (Bookmarked Ventures)'], 'Notes': ['Creating User', 'Users All', 'PITCH Category', 'Note Type', 'Project', 'Event'], 'PITCH Categories': ['Mirror Tag', 'Users (Default Category)'], 'Folder Types': [], 'Network Categories': [], 'Stages': ['Project'], 'Documents': ['Folder'], 'Restaurants': [], 'Tasks': ['Assigned User (Main)', 'Creating User', 'Task Priority Level', 'Event', 'Calendar Item', 'Task Type', 'PITCH Category', 'Project'], 'Accounts': [], 'Task Progress status': [], 'Shopping List Items': [], 'Contacts': ['Account', 'Users All', 'Creating User', 'Events (All Contacts)', 'Tags All', 'Contact Type'], 'Platforms': ['Events (Initial Inquiry)', 'Mirr

In [49]:
unique_columns_dict["Users"]

['Events (Initial Contact)',
 'Tasks (Assigned Main)',
 'Tasks (Creator)',
 'Default Task Priority Level',
 'Projects (All users)',
 'Notes (Creating User)',
 'Notes (All Users)',
 'Projects (Creating User)',
 'Messages Sent',
 'Messages Liked',
 'Messages Seen',
 'Status',
 'Status Logs (User Created)',
 'Default Note Type- Event',
 'Default Task Type- Event',
 'Default PITCH Category',
 'Conversations (Users Involved)',
 'Conversations (Creating User)',
 'Conversation Logs (Creating User)',
 'Calendar Items (Creating User)',
 'Calendar Items (All Users)',
 'Default Calendar Item Type - Event',
 'Default Calendar Item Type - Task',
 'Default Calendar Item Type - Goal',
 'Contacts (Users All)',
 'Contacts (Creating User)',
 'Orders',
 'Default Restaurant',
 'Test Note',
 'Learning Pathways',
 'Industry Subcategories (Interested)',
 'Industries (Skills)',
 'Industry Subcategories (Skills)',
 'Goals (Creating User)',
 'Goals (All Users)',
 'User Types (Old)',
 'Service Requests Created',

In [30]:
def search_data(search_criteria, collection_data_frames, result=None, visited=None):
    if result is None:
        result = {}
    if visited is None:
        visited = set()
    
    if not search_criteria:
        return result
    
    current_search = search_criteria[0]
    collection_name = current_search['collection']
    search_key = current_search.get('key')
    search_value = current_search.get('value')
    columns = current_search.get('columns', ['id', 'Name'])
    
    if collection_name not in collection_data_frames:
        return result
    
    collection_df = collection_data_frames[collection_name]
    
    if search_key and search_value:
        search_results = collection_df[collection_df[search_key] == search_value]
    else:
        search_results = collection_df
    
    if len(search_results) == 0:
        return result
    
    for _, row in search_results.iterrows():
        row_data = row.to_dict()
        row_id = (collection_name, row_data['id'])
        if row_id not in visited:
            visited.add(row_id)
            item_data = {col: row_data.get(col) for col in columns if col in row_data}
            
            if len(search_criteria) > 1:
                next_searches = search_criteria[1:]
                next_search = next_searches[0]
                next_collection = next_search['collection']
                next_columns = next_search.get('columns', ['id', 'Name'])
    
                # Check for non-plural version of the column name
                next_collection_singular = next_collection[:-1] if next_collection.endswith('s') else next_collection
                if next_collection_singular in row_data:
                    next_search_ids = row_data[next_collection_singular]
                elif next_collection in row_data:
                    next_search_ids = row_data[next_collection]
                else:
                    next_search_ids = None
                
                if next_search_ids is not None:
                    if isinstance(next_search_ids, list):
                        item_data[next_collection] = []
                        for search_id in next_search_ids:
                            next_search_criteria = [{'collection': next_collection, 'key': 'id', 'value': search_id, 'columns': next_columns}] + next_searches[1:]
                            next_result = search_data(next_search_criteria, collection_data_frames, {}, visited)
                            if next_result:
                                item_data[next_collection].append(next_result[next_collection][0])
                    else:
                        next_search_criteria = [{'collection': next_collection, 'key': 'id', 'value': next_search_ids, 'columns': next_columns}] + next_searches[1:]
                        next_result = search_data(next_search_criteria, collection_data_frames, {}, visited)
                        if next_result:
                            item_data[next_collection] = next_result[next_collection][0]
            
            if collection_name not in result:
                result[collection_name] = [item_data]
            else:
                result[collection_name].append(item_data)
    
    return result

In [33]:
search_criteria = [
  {"collection": "Users", "key": "Full Name", "value": "Willie Barron", "columns": ["Tasks (Creator)", "Full Name"]},
]

search_criteria = [
  {'collection': 'Users', 'key': 'Full Name', 'value': 'Willie Barron', 'columns': ['id', 'Full Name']},
  {'collection': 'Tasks', 'columns': ['id']},
  # {'collection': 'Messages', 'columns': ['id', 'Text']},
]



result = search_data(search_criteria, collection_data_frames)
result = json.dumps(result, indent=2)
print(result)

{
  "Users": [
    {
      "id": 2,
      "Full Name": "Willie Barron"
    }
  ]
}
